In [2]:
from google.colab import files
uploaded = files.upload()

Saving 01000_lr.dat to 01000_lr.dat
Saving 01000_lr.hea to 01000_lr.hea
Saving 01001_lr.dat to 01001_lr.dat


In [5]:
from google.colab import files
uploaded = files.upload()

Saving scp_statements.csv to scp_statements.csv


In [11]:
from google.colab import files
uploaded = files.upload()

Saving ptbxl_database.csv to ptbxl_database (1).csv


In [25]:
#A4
import pandas as pd
import numpy as np
import ast
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
# ---------- Functions ----------
def load_ecg_metadata_for_clustering(ecg_csv_path, scp_csv_path, feature_columns):
    """
    Loads PTB-XL metadata, extracts selected features for clustering.
    """
    ecg_df = pd.read_csv(ecg_csv_path)
    scp_df = pd.read_csv(scp_csv_path)

    # Rename first column to 'scp_code' (it's unnamed in PhysioNet CSV)
    scp_df.rename(columns={scp_df.columns[0]: 'scp_code'}, inplace=True)

    # Create mapping dictionary from scp_code → diagnostic_class
    scp_mapping = scp_df.set_index('scp_code')['diagnostic_class'].to_dict()

    # Map scp_codes JSON to main diagnostic superclass
    def map_to_superclass(scp_codes_str):
        scp_codes = ast.literal_eval(scp_codes_str)
        if len(scp_codes) == 0:
            return None
        main_code = max(scp_codes, key=scp_codes.get)
        return scp_mapping.get(main_code, None)

    ecg_df['diagnosis'] = ecg_df['scp_codes'].apply(map_to_superclass)

    # Drop rows with missing features
    ecg_df = ecg_df.dropna(subset=feature_columns)

    # Extract only features for clustering (target removed)
    X = ecg_df[feature_columns].values
    return X


def scale_features(X):
    """
    Scales features using StandardScaler for better K-Means performance.
    """
    scaler = StandardScaler()
    return scaler.fit_transform(X)


def perform_kmeans_clustering(X, n_clusters=2, random_state=0):
    """
    Performs K-Means clustering and returns labels and cluster centers.
    """
    kmeans = KMeans(n_clusters=n_clusters, random_state=random_state, n_init="auto")
    kmeans.fit(X)
    return kmeans.labels_, kmeans.cluster_centers_


# ---------- Main Program ----------

if __name__ == "__main__":
    # Step 1: Select features (no target variable)
    feature_columns = ['age', 'height', 'weight']

    # Step 2: Load data for clustering
    X = load_ecg_metadata_for_clustering("ptbxl_database.csv", "scp_statements.csv", feature_columns)

    # Step 3: Scale features
    X_scaled = scale_features(X)

    # Step 4: Perform K-Means clustering
    labels, centers = perform_kmeans_clustering(X_scaled, n_clusters=2, random_state=0)

    # Step 5: Print results
    print("=== K-Means Clustering Results ===")
    print("Cluster Labels:")
    print(labels)
    print("\nCluster Centers (scaled):")
    print(centers)


=== K-Means Clustering Results ===
Cluster Labels:
[0 0 0 ... 0 1 0]

Cluster Centers (scaled):
[[-0.23998167  0.7469959   0.70818873]
 [ 0.22429479 -0.69816704 -0.66189658]]
